In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report

In [2]:
# Set random seed for reproducibility
np.random.seed(42)

# --- NeuralNetwork Class ---
class NeuralNetwork:
    """
    Neural Network model with L hidden layers and a single output (y)
    """

    def __init__(self, layer_sizes, learning_rate=0.01):
        self.layer_sizes = layer_sizes
        self.learning_rate = learning_rate

        self.weights = []
        self.biases = []
        self.num_layers = len(self.layer_sizes) - 1

        self.initialize_network()

    def initialize_network(self):
        """Initializes weights and biases for all layers."""
        for i in range(self.num_layers):
            input_dim = self.layer_sizes[i]
            output_dim = self.layer_sizes[i+1]

            # Xavier initialization for parameters
            scale_w = np.sqrt(2.0 / (input_dim + output_dim))
            self.weights.append(np.random.normal(loc=0, scale=scale_w, size=(input_dim, output_dim)))
            self.biases.append(np.random.normal(loc=0, scale=0.01, size=(1, output_dim)))

        self.n_params = sum(w.size for w in self.weights) + sum(b.size for b in self.biases)

    def sigmoid(self, X):
        """Sigmoid activation function."""
        return 1 / (1 + np.exp(-X))

    def softmax(self, X):
        """Softmax activation function."""
        exp_X = np.exp(X - np.max(X, axis=-1, keepdims=True))
        return exp_X / np.sum(exp_X, axis=-1, keepdims=True)

    def forward_pass(self, X):
        """Performs a forward pass and stores intermediate activations and Z-values."""
        self.activations = [X]
        self.zs = []

        for i in range(self.num_layers):
            current_input = self.activations[-1]
            weights_l = self.weights[i]
            biases_l = self.biases[i]

            Z = np.dot(current_input, weights_l) + biases_l
            self.zs.append(Z)

            if i < self.num_layers - 1: # Hidden layers use sigmoid
                A = self.sigmoid(Z)
            else: # Output layer uses softmax
                A = self.softmax(Z)
            self.activations.append(A)

        return self.activations[-1]

    def backward_pass(self, X, Y):
        """Computes gradients and updates weights/biases using backpropagation."""
        output_activations = self.activations[-1]
        delta = output_activations - Y # Delta for last layer
        deltas = [delta]

        # Propagate deltas backwards through hidden layers
        for i in range(self.num_layers - 1, 0, -1):
            W_l = self.weights[i]
            Z_prev = self.zs[i-1]

            delta = (deltas[-1] @ W_l.T) * (self.sigmoid(Z_prev) * (1 - self.sigmoid(Z_prev)))
            deltas.append(delta)

        deltas.reverse() # Reverse to match forward order (delta[0] for first layer, etc.)

        # Update weights and biases
        for i in range(self.num_layers):
            input_to_weights = self.activations[i]
            weights_delta = np.dot(input_to_weights.T, deltas[i])
            biases_delta = np.sum(deltas[i], axis=0, keepdims=True) # Sum delta over samples for biases

            self.weights[i] -= self.learning_rate * weights_delta
            self.biases[i] -= self.learning_rate * biases_delta

    def encode(self):
        """Encodes all model parameters (weights and biases) into a single 1D vector."""
        all_params = []
        for w in self.weights:
            all_params.append(w.ravel())
        for b in self.biases:
            all_params.append(b.ravel())
        theta = np.concatenate(all_params)
        return theta

    def decode(self, theta):
        """Decodes a 1D parameter vector into the model's weights and biases."""
        decoded_weights = []
        decoded_biases = []
        offset = 0

        for i in range(self.num_layers):
            input_dim = self.layer_sizes[i]
            output_dim = self.layer_sizes[i+1]

            # Decode weights
            weight_size = input_dim * output_dim
            W_l = theta[offset : offset + weight_size].reshape(input_dim, output_dim)
            decoded_weights.append(W_l)
            offset += weight_size

            # Decode biases
            bias_size = output_dim
            B_l = theta[offset : offset + bias_size].reshape(1, output_dim)
            decoded_biases.append(B_l)
            offset += bias_size

        self.weights = decoded_weights
        self.biases = decoded_biases

    def evaluate_proposal(self, theta, X_data):
        """
        Evaluates the network's output for given data using a specific parameter vector 'theta'.
        """
        # Temporarily decode theta into local weights/biases for evaluation
        temp_weights = []
        temp_biases = []
        offset = 0

        for i in range(self.num_layers):
            input_dim = self.layer_sizes[i]
            output_dim = self.layer_sizes[i+1]

            weight_size = input_dim * output_dim
            W_l = theta[offset : offset + weight_size].reshape(input_dim, output_dim)
            temp_weights.append(W_l)
            offset += weight_size

            bias_size = output_dim
            B_l = theta[offset : offset + bias_size].reshape(1, output_dim)
            temp_biases.append(B_l)
            offset += bias_size

        # Perform forward pass using temp_weights and temp_biases
        activations = [X_data]
        for i in range(self.num_layers - 1): # Hidden layers
            current_input = activations[-1]
            Z = np.dot(current_input, temp_weights[i]) + temp_biases[i]
            A = self.sigmoid(Z)
            activations.append(A)

        # Output layer
        final_input = activations[-1]
        Z_out = np.dot(final_input, temp_weights[-1]) + temp_biases[-1]
        fx = self.softmax(Z_out)

        return fx

    def langevin_gradient(self, x_data, y_data, theta, depth):
        """
        Computes a gradient-based proposal for MCMC.
        Temporarily applies `theta` to the model, performs SGD steps, then encodes and returns.
        """
        # Save current model state
        original_weights = [w.copy() for w in self.weights]
        original_biases = [b.copy() for b in self.biases]

        # Apply the input theta to the model
        self.decode(theta)

        # Perform SGD steps
        for _ in range(0, depth):
            self.forward_pass(x_data)
            self.backward_pass(x_data, y_data)

        # Get the updated theta
        theta_updated = self.encode()

        # Restore original model state
        self.weights = original_weights
        self.biases = original_biases

        return theta_updated



In [3]:
# --- MCMC Class ---
class MCMC:
    def __init__(self, n_samples, n_burnin, x_data, y_data, x_test, y_test, layer_sizes):
        self.n_samples = n_samples
        self.n_burnin = n_burnin
        self.x_data = x_data
        self.y_data = y_data
        self.x_test = x_test
        self.y_test = y_test

        self.step_theta = 0.025 # Step size for the MCMC proposal (also called  std dev)
        self.sigma_squared = 25 # Variance for the Gaussian prior on parameters

        self.model = NeuralNetwork(layer_sizes)
        self.theta_size = self.model.n_params

        self.use_langevin_gradients = True # whether to use langevin or not
        self.sgd_depth = 1 # Number of SGD steps for Langevin gradient
        self.l_prob = 0.5 # Probability of using Langevin proposal vs. simple random walk

        # Storage for posterior samples and predictions
        self.pos_theta = None
        self.pred_y = None
        self.sim_y = None
        self.accuracy_data = None
        self.test_pred_y = None
        self.test_sim_y = None
        self.test_accuracy_data = None

    @staticmethod
    def accuracy(predictions, targets):
        """Calculates classification accuracy."""
        predicted_classes = np.argmax(predictions, axis=1)
        if targets.ndim > 1 and targets.shape[1] > 1: # as sometime people do not OHE the output
            true_classes = np.argmax(targets, axis=1)
        else: # Assuming  targets are already integer labels or flattened 1D array
            true_classes = targets.flatten()

        return accuracy_score(true_classes, predicted_classes) * 100

    def likelihood_function(self, theta, test=False):
        """Calculates the multinomial log-likelihood (cross-entropy) and accuracy."""
        x_data_eval = self.x_test if test else self.x_data
        y_data_eval = self.y_test if test else self.y_data

        model_prediction = self.model.evaluate_proposal(theta, x_data_eval)
        model_prediction = np.maximum(model_prediction, 1e-12) # Clipping  to prevent log(0) and stability

        log_likelihood = np.sum(y_data_eval * np.log(model_prediction))
        accuracy = self.accuracy(model_prediction, y_data_eval)
        model_simulation = model_prediction # we can use it later for data generation

        return log_likelihood, model_prediction, model_simulation, accuracy

    def prior(self, sigma_squared, theta):
        """Calculates the log-prior probability for parameters (Gaussian prior)."""
        part1 = -self.model.n_params / 2.0 * np.log(sigma_squared)
        part2 = 1 / (2.0 * sigma_squared) * (np.sum(np.square(theta)))
        log_prior = part1 - part2
        return log_prior

    def MCMC_sampler(self):
        """Runs the MCMC sampler to generate posterior samples."""
        output_dim = self.model.layer_sizes[-1]
        pos_theta = np.zeros((self.n_samples, self.theta_size))
        pred_y = np.zeros((self.n_samples, self.x_data.shape[0], output_dim))
        test_pred_y = np.zeros((self.n_samples, self.x_test.shape[0], output_dim))
        sim_y = np.zeros((self.n_samples, self.x_data.shape[0], output_dim))
        test_sim_y = np.zeros((self.n_samples, self.x_test.shape[0], output_dim))
        accuracy_data = np.zeros(self.n_samples)
        test_accuracy_data = np.zeros(self.n_samples)

        # Initialisation: starting point for the chain
        theta = np.random.randn(self.theta_size) # Initial random parameters

        # Calculate initial likelihood and prior
        prior_val = self.prior(self.sigma_squared, theta)
        (likelihood, initial_train_pred, _, accuracy_train) = self.likelihood_function(theta, test=False)
        (test_likelihood, initial_test_pred, _, accuracy_test) = self.likelihood_function(theta, test=True)

        pos_theta[0, :] = theta
        pred_y[0, :, :] = initial_train_pred
        sim_y[0, :, :] = initial_train_pred
        accuracy_data[0] = accuracy_train
        test_pred_y[0, :, :] = initial_test_pred
        test_sim_y[0, :, :] = initial_test_pred
        test_accuracy_data[0] = accuracy_test

        n_accepted_samples = 0

        print(f"MCMC Chain started for {self.n_samples} samples...")
        print(f"Initial Train Acc: {accuracy_train:.2f}%, Test Acc: {accuracy_test:.2f}%")

        for ii in range(1, self.n_samples):
            theta_current = pos_theta[ii - 1, :]
            prior_current = prior_val
            likelihood_current = likelihood

            lx = np.random.uniform(0, 1)
            if lx < self.l_prob and self.use_langevin_gradients:
                theta_gd = self.model.langevin_gradient(self.x_data, self.y_data, theta_current, self.sgd_depth)
                theta_proposal = np.random.normal(theta_gd, self.step_theta, self.theta_size)
            else:
                theta_proposal = np.random.normal(theta_current, self.step_theta, self.theta_size)

            prior_proposal = self.prior(self.sigma_squared, theta_proposal)
            (likelihood_proposal, current_train_pred, _, accuracy_train_proposal) = self.likelihood_function(theta_proposal, test=False)
            (test_likelihood_proposal, current_test_pred, _, accuracy_test_proposal) = self.likelihood_function(theta_proposal, test=True)

            diff_likelihood = likelihood_proposal - likelihood_current
            diff_prior = prior_proposal - prior_current

            diff_prop = 0.0

            if lx < self.l_prob and self.use_langevin_gradients:
                theta_gd_reverse = self.model.langevin_gradient(self.x_data, self.y_data, theta_proposal, self.sgd_depth)

                log_q_prop_given_curr = -0.5 * np.sum(np.square(theta_proposal - theta_gd)) / (self.step_theta**2)
                log_q_curr_given_prop = -0.5 * np.sum(np.square(theta_current - theta_gd_reverse)) / (self.step_theta**2)

                diff_prop = log_q_curr_given_prop - log_q_prop_given_curr

            mh_prob = np.min([1.0, np.exp(diff_likelihood + diff_prior + diff_prop)])

            u = np.random.uniform(0, 1)

            if u < mh_prob:
                pos_theta[ii, :] = theta_proposal
                likelihood = likelihood_proposal
                prior_val = prior_proposal
                pred_y[ii, :, :] = current_train_pred
                sim_y[ii, :, :] = current_train_pred
                accuracy_data[ii] = accuracy_train_proposal
                test_pred_y[ii, :, :] = current_test_pred
                test_sim_y[ii, :, :] = current_test_pred
                test_accuracy_data[ii] = accuracy_test_proposal
                n_accepted_samples += 1
            else:
                pos_theta[ii, :] = pos_theta[ii - 1, :]
                pred_y[ii, :, :] = pred_y[ii - 1, :, :]
                sim_y[ii, :, :] = sim_y[ii - 1, :, :]
                accuracy_data[ii] = accuracy_data[ii - 1]
                test_pred_y[ii, :, :] = test_pred_y[ii - 1, :, :]
                test_sim_y[ii, :, :] = test_sim_y[ii - 1, :, :]
                test_accuracy_data[ii] = test_accuracy_data[ii - 1]

            if (ii + 1) % 500 == 0:
                acceptance_rate = (n_accepted_samples / (ii + 1)) * 100
                print(f"Sample {ii+1}/{self.n_samples} | Accept Rate: {acceptance_rate:.2f}% | "
                      f"Train Acc: {accuracy_data[ii]:.2f}% | Test Acc: {test_accuracy_data[ii]:.2f}%")

        print("MCMC sampling complete. Applying burn-in.")

        self.pos_theta = pos_theta[self.n_burnin:, :]
        self.pred_y = pred_y[self.n_burnin:, :, :]
        self.sim_y = sim_y[self.n_burnin:, :, :]
        self.accuracy_data = accuracy_data[self.n_burnin:]
        self.test_pred_y = test_pred_y[self.n_burnin:, :, :]
        self.test_sim_y = test_sim_y[self.n_burnin:, :, :]
        self.test_accuracy_data = test_accuracy_data[self.n_burnin:]

        results_df = pd.DataFrame(self.pos_theta, columns=[f"theta_{i}" for i in range(self.theta_size)])
        return results_df

In [4]:

# --- Data Preparation and Execution ---

# Load Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# One-hot encode target labels
encoder = OneHotEncoder(sparse_output=False)
y_onehot = encoder.fit_transform(y.reshape(-1, 1))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

# Define network architecture
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]
hidden_layer_nodes = [10, 5]
layer_sizes = [input_dim] + hidden_layer_nodes + [output_dim]

print(f"Network Architecture: {layer_sizes}")


Network Architecture: [4, 10, 5, 3]


In [5]:
# MCMC parameters
n_samples = 5000  # Number of MCMC samples
n_burnin = 1000   # Number of burn-in samples

# Initialize and run MCMC sampler
print("Starting MCMC sampling...")
bnn_mcmc = MCMC(n_samples, n_burnin, X_train, y_train, X_test, y_test, layer_sizes)
posterior_samples_df = bnn_mcmc.MCMC_sampler()
print("MCMC sampling finished.")

print("\nShape of posterior samples (excluding the burn-in period ):", posterior_samples_df.shape)
print("First 5 posterior samples:\n", posterior_samples_df.head())


Starting MCMC sampling...
MCMC Chain started for 5000 samples...
Initial Train Acc: 33.33%, Test Acc: 33.33%
Sample 500/5000 | Accept Rate: 45.80% | Train Acc: 98.33% | Test Acc: 96.67%
Sample 1000/5000 | Accept Rate: 47.60% | Train Acc: 98.33% | Test Acc: 100.00%
Sample 1500/5000 | Accept Rate: 46.53% | Train Acc: 98.33% | Test Acc: 100.00%
Sample 2000/5000 | Accept Rate: 45.90% | Train Acc: 98.33% | Test Acc: 96.67%
Sample 2500/5000 | Accept Rate: 46.88% | Train Acc: 99.17% | Test Acc: 100.00%
Sample 3000/5000 | Accept Rate: 47.27% | Train Acc: 98.33% | Test Acc: 96.67%
Sample 3500/5000 | Accept Rate: 47.40% | Train Acc: 99.17% | Test Acc: 96.67%
Sample 4000/5000 | Accept Rate: 46.75% | Train Acc: 100.00% | Test Acc: 93.33%
Sample 4500/5000 | Accept Rate: 47.02% | Train Acc: 100.00% | Test Acc: 96.67%
Sample 5000/5000 | Accept Rate: 47.44% | Train Acc: 100.00% | Test Acc: 96.67%
MCMC sampling complete. Applying burn-in.
MCMC sampling finished.

Shape of posterior samples (excluding t

In [6]:
# --- Evaluate Performance ---
print("\n Final Performance Evaluation")

# Convert one-hot encoded true labels to integer labels for accuracy_score
true_train_labels = np.argmax(y_train, axis=1)
true_test_labels = np.argmax(y_test, axis=1)

# 1. Using the mean of predictions from all accepted posterior samples (Posterior Predictive Mean)
# This averages the probabilities across all samples, we will take argmax
mean_train_predictions_prob = np.mean(bnn_mcmc.pred_y, axis=0)
mean_test_predictions_prob = np.mean(bnn_mcmc.test_pred_y, axis=0)

train_predicted_labels_mean_pred = np.argmax(mean_train_predictions_prob, axis=1)
test_predicted_labels_mean_pred = np.argmax(mean_test_predictions_prob, axis=1)

final_train_accuracy_mean_pred = accuracy_score(true_train_labels, train_predicted_labels_mean_pred) * 100
final_test_accuracy_mean_pred = accuracy_score(true_test_labels, test_predicted_labels_mean_pred) * 100

print(f"\nFinal Train Accuracy (from posterior predictive mean): {final_train_accuracy_mean_pred:.2f}%")
print(f"Final Test Accuracy (from posterior predictive mean): {final_test_accuracy_mean_pred:.2f}%")

print("\nTest Classification Report (from posterior predictive mean):")
print(classification_report(true_test_labels, test_predicted_labels_mean_pred, target_names=iris.target_names))

# 2. Average of individual sample accuracies
avg_sampled_train_accuracy = np.mean(bnn_mcmc.accuracy_data)
avg_sampled_test_accuracy = np.mean(bnn_mcmc.test_accuracy_data)
print(f"\nAverage Train Accuracy (across accepted samples): {avg_sampled_train_accuracy:.2f}%")
print(f"Average Test Accuracy (across accepted samples): {avg_sampled_test_accuracy:.2f}%")


 Final Performance Evaluation

Final Train Accuracy (from posterior predictive mean): 99.17%
Final Test Accuracy (from posterior predictive mean): 96.67%

Test Classification Report (from posterior predictive mean):
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       0.90      1.00      0.95         9
   virginica       1.00      0.91      0.95        11

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30


Average Train Accuracy (across accepted samples): 99.10%
Average Test Accuracy (across accepted samples): 96.63%


## SKLEARN RESULTS


In [12]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neural_network import MLPClassifier # The frequentist neural network
from sklearn.metrics import accuracy_score, classification_report

np.random.seed(42)

# --- Data Preparation ---
iris = load_iris()
X, y_int = iris.data, iris.target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

encoder = OneHotEncoder(sparse_output=False)
y_ohe = encoder.fit_transform(y_int.reshape(-1, 1))

# Split data
X_train, X_test, y_train_int, y_test_int = train_test_split(X_scaled, y_int, test_size=0.2, random_state=42)

# --- Neural Network Model (Frequentist) ---
print("--- Training Frequentist Neural Network ---")

mlp_model = MLPClassifier(hidden_layer_sizes=(10, 5),
                          activation='relu',
                          solver='adam',
                          alpha=0.0001, # L2 regularization (weight decay)
                          max_iter=5000, # Max training epochs
                          random_state=42,
                          verbose=False)

# Train the model
mlp_model.fit(X_train, y_train_int)


# --- Evaluation ---
print("\n--- Evaluating Frequentist Neural Network Performance ---")

# Predictions
train_pred_int = mlp_model.predict(X_train)
test_pred_int = mlp_model.predict(X_test)

# Accuracy
train_accuracy = accuracy_score(y_train_int, train_pred_int) * 100
test_accuracy = accuracy_score(y_test_int, test_pred_int) * 100

print(f"Train Accuracy: {train_accuracy:.2f}%")
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Classification Report
print("\nTest Classification Report:")
print(classification_report(y_test_int, test_pred_int, target_names=iris.target_names))

print("\n--- Frequentist Neural Network Performance Evaluation Complete ---")

--- Training Frequentist Neural Network ---

--- Evaluating Frequentist Neural Network Performance ---
Train Accuracy: 98.33%
Test Accuracy: 100.00%

Test Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      1.00      1.00         9
   virginica       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30


--- Frequentist Neural Network Performance Evaluation Complete ---
